# Import Dependencies

In [85]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import numpy as np
from scipy.stats import boxcox

# Class: LoadData

Used for loading data after the `run_table`.

In [86]:
class LoadData:

    __num_rows = 0
    __s_folder = None
    __algo = None

    def __init__(self, num_rows: int, s_folder: str, algo: str):
        self.__num_rows = num_rows
        self.__s_folder = s_folder
        self.__algo = algo

    def load_run_table(self):
        try:
            run_table = pd.read_csv(f'{self.__s_folder}/run_table.csv', nrows=self.__num_rows)
            return run_table
        except:
            return None
        
    def log_transform(self, df, label):
        return df[label].apply(lambda x: np.log(x + 1) if x > 0 else np.nan)

    def boxcox_transform(self, df, label):
        return boxcox(df[label].dropna() + 1)
    
    def filter_df(self, df, label, value):
        return df[df[label] == value]
    
    def group_df_by(self, df, key, outliers):
        grouped_df = df.groupby(key)
        if not outliers:
            return grouped_df
        else:
            cleaned_df = grouped_df.apply(self.remove_outliers, column_name='avg_energy_pct')
            cleaned_df = cleaned_df.rename(columns={key: f'new_{key}'})
            #cleaned_df.reset_index()
            regrouped_df = cleaned_df.groupby(key)
            return regrouped_df
            
    def load_nav2_success(self, component, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        energy_files = {}
        sum_success_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            if row['__done'] == 'TODO':
                continue
            folder_path = f"{self.__s_folder}/{run_id}"
            file_path = os.path.join(folder_path, 'nav2_performance.csv')
            if file_path:
                try:
                    nav2_df = pd.read_csv(file_path)
                    nav2_df['success'] = pd.to_numeric(nav2_df['success'])
                    success = nav2_df['success'].sum()
                    runs_data[run_id] = success
                except Exception as e:
                    print(f"Error processing file for run_id {run_id}: {e}")
                    runs_data[run_id] = 0

        sum_success_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'success'])
        merged_df = df.merge(sum_success_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df
    
    def load_nav2_time(self, component, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        nav_time_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            if row['__done'] == 'TODO':
                continue
            folder_path = f"{self.__s_folder}/{run_id}"
            file_path = os.path.join(folder_path, 'nav2_performance.csv')
            if file_path:
                try:
                    nav2_df = pd.read_csv(file_path)
                    nav2_df['navigation_time'] = pd.to_numeric(nav2_df['navigation_time'], errors='coerce')
                    nav_time = nav2_df['navigation_time'].sum()
                    if nav_time == 0:
                        nav_time = 100
                    runs_data[run_id] = nav_time
                except Exception as e:
                    # print(f"Error processing file for run_id {run_id}: {e}")
                    runs_data[run_id] = 100

        nav_time_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'navigation_time'])
        merged_df = df.merge(nav_time_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df

    def load_power(self, component, transform: bool, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        energy_files = {}
        avg_energy_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            folder_path = f"{self.__s_folder}/{run_id}"
            energy_files = glob.glob(os.path.join(folder_path, f'pj_{component}_server.csv-*.csv'))
            if energy_files:
                try:
                    energy_df = pd.read_csv(energy_files[0])
                    energy_df['CPU Power'] = pd.to_numeric(energy_df['CPU Power'], errors='coerce')
                    if transform:
                        energy_df['CPU Power'], _ = self.boxcox_transform(energy_df, 'CPU Power')
                    filtered_df = energy_df[energy_df['CPU Power'] > 0.001]
                    avg_energy_pct = filtered_df['CPU Power'].mean()
                    
                    runs_data[run_id] = avg_energy_pct
                except Exception as e:
                    print(f"Error processing file for run_id {run_id}: {e}")

        avg_energy_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'avg_energy_pct'])
        merged_df = df.merge(avg_energy_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df

    def remove_outliers(self, group, column_name):
        Q1 = group[column_name].quantile(0.25)
        Q3 = group[column_name].quantile(0.75)
        
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return group[(group[column_name] >= lower_bound) & (group[column_name] <= upper_bound)]

    def load_all_energy():
        pass

    def load_energy_mean():
        pass

    def load_energy_total():
        pass

    def load_all_cpu():
        pass

    def load_all_memory():
        pass

# Setting Arguments

Like source and destination folders.

In [87]:
# Paths
prefix = "data/greenros_reconf_world_small_full/"
s_folder = "data/greenros_reconf_world_small_full/"
std_folder = s_folder
d_folder = "./graphs/"

# Ensure the output folder exists
os.makedirs(d_folder, exist_ok=True)

# Loading Data

Loads all the CSVs.

In [88]:

# Load Runtable
load = LoadData(num_rows=175, s_folder=s_folder, algo="all")

# Plotting Methods

All the methods used for plotting the data.

In [89]:
# Boxplot for Power Consumption
def gen_power_boxplot(component="controller", group_by="configuration", filter=None, transform=False, outliers=False ):
    data = load.load_power(component, transform, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    grouped = data.groupby(group_by)

    print(data.head())

    boxplot_data = []
    labels = []

    for group_name, group_data in grouped:
        boxplot_data.append(group_data["avg_energy_pct"].values)
        labels.append(group_name)

    # Plot using Seaborn
    plt.figure(figsize=(4, 4))
    sns.boxplot(data=boxplot_data)
    plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
    plt.ylabel("Power (W)")
    plt.xlabel(group_by.capitalize())
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"boxplot_{component}_{group_by}_power.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")

In [79]:
# Boxplot for Nav2 Navigation Time
def gen_nav_time_boxplot(component="controller", group_by="configuration", filter=None, outliers=False ):
    data = load.load_nav2_time(component, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    grouped = data.groupby(group_by)

    print(data.head())

    boxplot_data = []
    labels = []

    for group_name, group_data in grouped:
        boxplot_data.append(group_data["navigation_time"].values)
        labels.append(group_name)

    # Plot using Seaborn
    plt.figure(figsize=(4, 4))
    sns.boxplot(data=boxplot_data)
    plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
    plt.ylabel("Navigation Time (s)")
    plt.xlabel(group_by.capitalize())
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"boxplot_{component}_{group_by}_nav_time.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")

In [58]:
# Boxplot for Nav2 Success Rate
def gen_nav2_success_barplot(component="controller", group_by="configuration", filter=None, outliers=False):
    data = load.load_nav2_success(component, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    # Group and sum success counts
    grouped = data.groupby(group_by)["success"].sum().reset_index()

    print(grouped.head())

    # Plot using Seaborn
    plt.figure(figsize=(5, 4))
    sns.barplot(x=group_by, y="success", data=grouped)
    plt.xticks(rotation=90)
    plt.ylabel("Total Success")
    plt.xlabel(group_by.capitalize())
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"barplot_{component}_{group_by}_success.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")


In [ ]:
# Boxplot for Nav2 Distance Traveled

In [ ]:
# Boxplot for Nav2 Recoveries

# Controller

In [92]:
component = "planner"

In [93]:
gen_power_boxplot(component, group_by="configuration", filter = ("number_obstacles", 2), transform=False, outliers=False)

  __run_id __done  round  configuration  position_goal  number_obstacles  \
1    run_1   DONE      0              0              2                 2   
3    run_3   DONE      0              1              2                 2   
5    run_5   DONE      0              2              2                 2   
7    run_7   DONE      0              3              2                 2   
9    run_9   DONE      0              4              2                 2   

   avg_energy_pct  
1        0.001362  
3        0.001551  
5        0.001852  
7        0.001541  
9        0.001750  
Saved: ./graphs/boxplot_planner_configuration_power.pdf


In [59]:
gen_nav2_success_barplot(component, group_by="configuration", filter = None, outliers=False)

Error processing file for run_id run_16: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_16/nav2_performance.csv'
Error processing file for run_id run_22: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_22/nav2_performance.csv'
Error processing file for run_id run_40: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_40/nav2_performance.csv'
Error processing file for run_id run_106: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_106/nav2_performance.csv'
Error processing file for run_id run_138: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_138/nav2_performance.csv'
   configuration  success
0              0        7
1              1        8
2              2        6
3              3        8
4              4        8
Saved: ./graphs/barplot_planner_configuration_success.pdf


In [84]:
gen_nav2_success_barplot(component, group_by="configuration", filter = None, outliers=False)

Error processing file for run_id run_16: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_16/nav2_performance.csv'
Error processing file for run_id run_22: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_22/nav2_performance.csv'
Error processing file for run_id run_40: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_40/nav2_performance.csv'
Error processing file for run_id run_106: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_106/nav2_performance.csv'
Error processing file for run_id run_138: [Errno 2] No such file or directory: 'data/greenros_reconf_world_small_full//run_138/nav2_performance.csv'
   configuration  success
0              0        7
1              1        8
2              2        6
3              3        8
4              4        8
Saved: ./graphs/barplot_planner_configuration_success.pdf


In [83]:
gen_nav_time_boxplot(component, group_by="configuration", filter = None, outliers=False)

  __run_id __done  round  configuration  position_goal  number_obstacles  \
0    run_0   DONE      0              0              2                 0   
1    run_1   DONE      0              0              2                 2   
2    run_2   DONE      0              1              2                 0   
3    run_3   DONE      0              1              2                 2   
4    run_4   DONE      0              2              2                 0   

   navigation_time  
0               46  
1               39  
2               50  
3               52  
4               35  
Saved: ./graphs/boxplot_planner_configuration_nav_time.pdf
